In [ ]:
import tensorflow as tf
import numpy as np
import os

In [ ]:
# The SegNet Architechture

class SegNet:
    
    def __init__(self, image): 
        
        
    def encoder(self, image):
        ''' Encoder Network for SegNet
        - 2 convolutions of 64 3x3 kernels with relu activation
        - 2x2 max pooling
        - 2 convolutions of 128 3x3 kernels with relu activation
        - 2x2 max pooling
        - 3 convolutions of 256 3x3 kernels with relu activation
        - 2x2 max pooling
        - 3 convolutions of 512 3x3 kernels with relu activation
        - 2x2 max pooling
        - 3 convolutions of 512 3x3 kernels with relu activation
        - 2x2 max pooling
        '''
        
        # Ensure input is of shape [Batchsize, Height, Width, Channels]
        input_layer = tf.reshape(image, [-1, 224, 224, 3])
        
        # store maxPool indices as a list
#         self.poolIndices = {}
                
        # First conv layer
        self.conv1_1 = conv_relu(input_layer, 64)
        self.conv1_2 = conv_relu(conv1_1, 64)
        self.pool1 = maxpool(conv1_2)
        
#         self.poolIndices.append(max1)
        
        # Second conv layer
        self.conv2_1 = conv_relu(pool1, 128)
        self.conv2_2 = conv_relu(conv2_1, 128)
        self.pool2 = maxpool(conv2_2)

#         self.poolIndices.append(max2)
        
        # Third conv layer
        self.conv3_1 = conv_relu(pool2, 256)
        self.conv3_2 = conv_relu(conv3_1, 256)
        self.conv3_3 = conv_relu(conv3_2, 256)  
        self.pool3 = maxpool(conv3_3)
        
#         self.poolIndices.append(max3)
        
        # Fourth conv layer
        self.conv4_1 = conv_relu(pool3, 512)
        self.conv4_2 = conv_relu(conv4_1, 512)
        self.conv4_3 = conv_relu(conv4_2, 512)  
        self.pool4 = maxpool(conv4_3)
        
#         self.poolIndices.append(max4)
        
        # Fifth conv layer
        self.conv5_1 = conv_relu(pool4, 512)
        self.conv5_2 = conv_relu(conv5_1, 512)
        self.conv5_3 = conv_relu(conv5_2, 512)  
        self.pool5 = maxpool(conv5_3)

#         self.poolIndices.append(max5)
        return
    
    def decoder(self):
        deconv5_3 = transposeconv_relu(self.conv5_3, 512)
        deconv5_2 = transposeconv_relu(deconv5_3, 512)
        deconv5_1 = transposeconv_relu(deconv5_2, 512)


In [ ]:
# Helper functions

def conv_relu(image, num_filters, ksize=[3,3], stride=1, pad="SAME"):
    
    conv = tf.layers.conv2d(inputs=image, filters=num_filters, kernel_size=ksize, 
                                strides=stride, padding=pad, activation=tf.nn.relu, 
                                kernel_initializer=tf.contrib.layers.xavier_initializer)
    return conv

def maxpool(image, kernel=[1,2,2,1], stride=[1,2,2,1], pad="VALID"):
    pool = tf.nn.max_pool(value=image, ksize=kernel, strides=stride, padding=pad)
#     pool, maxargs = tf.nn.max_pool_with_argmax(value=image, ksize=kernel, strides=stride, padding=pad)
    return pool

def transposeconv_relu(image, num_filters, ksize=[3,3], stride=1, pad="SAME"):
    conv_transpose = tf.layers.conv2d_transpose(inputs=image, filters=num_filters, kernel_size=ksize,
                                                    strides=stride, padding=pad, activation=tf.nn.relu,
                                                     kernel_initializer=tf.contrib.layers.xavier_initializer)
    
    return conv_transpose